In [1]:
import numpy as np
import pandas as pd

In [2]:
listing=pd.read_csv('C:\\Users\Windows\_ANCONDA WORKSHOP\_VAMSHI SIR\datasets\listings.csv')

In [5]:
listing.head(1)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,7087327,https://www.airbnb.com/rooms/7087327,20151002231825,2015-10-03,Historic DC Condo-Walk to Capitol!,Professional pictures coming soon! Welcome to ...,NaN,Professional pictures coming soon! Welcome to ...,none,NaN,...,NaN,f,NaN,"DISTRICT OF COLUMBIA, WASHINGTON",f,flexible,f,f,18,NaN


In [6]:
listing.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary',
       'space', 'description', 'experiences_offered', 'neighborhood_overview',
       'notes', 'transit', 'thumbnail_url', 'medium_url', 'picture_url',
       'xl_picture_url', 'host_id', 'host_url', 'host_name', 'host_since',
       'host_location', 'host_about', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood',
       'host_listings_count', 'host_total_listings_count',
       'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
       'street', 'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market',
       'smart_location', 'country_code', 'country', 'latitude', 'longitude',
       'is_location_exact', 'property_type', 'room_type', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', '

In [7]:
listing.shape

(3723, 92)

In [8]:
import numpy as np
our_accommodates_value=3
first_living_space_value=listing.loc[0,'accommodates']
first_distance=np.abs(first_living_space_value-our_accommodates_value)

In [9]:
first_distance

1

In [10]:
first_living_space_value

4

In [35]:
#calculating distacne for whole columns in comparison to our acc value
#generting new column distance
listing['distance']=np.abs(listing.accommodates-our_accommodates_value)
listing.distance.value_counts().sort_index().head()

0     461
1    2294
2     503
3     279
4      35
Name: distance, dtype: int64

In [37]:
# shuffling the dataset and randomly selecting five values
random_list=listing.sample(frac=1,random_state=0)

In [39]:
random_list=random_list.sort_values('distance')
random_list.price.head()

2645     $75.00
2825    $120.00
2145     $90.00
2541     $50.00
3349    $105.00
Name: price, dtype: object

In [46]:
random_list.price=random_list.price.str.replace('\$|,','').astype(float)
mean_price=random_list.price.iloc[0:5].mean()
mean_price

88.0

In [47]:
random_list.price

2645      75.0
2825     120.0
2145      90.0
2541      50.0
3349     105.0
2228     117.0
1122     176.0
2311     117.0
625      150.0
2312     100.0
2419     119.0
2421     139.0
131      215.0
3343      99.0
976       95.0
2376     170.0
3625      80.0
2700     120.0
1499      94.0
103      249.0
670      121.0
2957     120.0
1715      90.0
2857      98.0
421       50.0
3340      80.0
230      289.0
2579     135.0
710       89.0
880      175.0
         ...  
542      340.0
529      600.0
1878      99.0
2995     595.0
3272     599.0
3305    2000.0
1326     375.0
1656     283.0
1875      99.0
808      215.0
571      720.0
3524     295.0
685      399.0
1403     599.0
562      599.0
1658     283.0
657      500.0
2688     749.0
1850     180.0
1806     330.0
2106     499.0
2757     245.0
611     1250.0
1818      10.0
1402    1200.0
763     1000.0
2560      60.0
1594    1250.0
1224     499.0
1596     299.0
Name: price, Length: 3723, dtype: float64

### TRAIN TEST APPROACH

In [56]:
listing.drop('distance',axis=1,inplace=True)

In [91]:
train_df=listing.copy().iloc[:2792]
test_df=listing.copy().iloc[2792:]

In [93]:
#the listing is already randomised so no need to do that
def predict_price(new_listing_value,feature_column):
    temp_df= train_df
    temp_df['distance']=np.abs(listing[feature_column]-new_listing_value)
    temp_df=temp_df.sort_values('distance')
    knn_5=temp_df.price.iloc[:5]
    predicted_price=knn_5.mean()
    return(predicted_price)


In [94]:
test_df['predicted_price']=test_df.accommodates.apply(predict_price,feature_column='accommodates')

In [95]:
predict_price(3,'accommodates')

177.4

In [96]:
test_df['squared_error']=(test_df['predicted_price']-test_df['price'])**(2)

In [100]:
mse=test_df['squared_error'].mean()
mse

18646.525370569325

In [101]:
rmse=mse**0.5
rmse

136.55228072269364

In [102]:
# $136.55228072269364 rmse

In [105]:
listofcol=['accommodates','bedrooms','bathrooms','number_of_reviews']
for feature in listofcol:
    test_df['predicted_price']=test_df.accommodates.apply(predict_price,feature_column=feature)
    test_df['squared_error']=(test_df['price']-test_df['predicted_price'])**(2)
    mse=test_df['squared_error'].mean()
    rmse=mse**0.5
    print('rmse value for {} column is {}'.format(feature,rmse))

rmse value for accommodates column is 136.55228072269364
rmse value for bedrooms column is 255.585704640843
rmse value for bathrooms column is 345.58305307635834
rmse value for number_of_reviews column is 145.4937932243503
